# LSTM Implementation

In [1]:
#
#  tratar nulos y outliers
# escalar los valores (en este caso creo que basta con min max scaler)
#  codificar la estacionalidad. A menudo se utilizan variables de fecha cíclicas (por ejemplo, seno/coseno del día del año) para capturar la estacionalidad anual.

In [2]:
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras import layers, Model, Sequential
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler


# df = pd.read_csv("data/stickers/train_preprocessed.csv")
# df;

## Arquitectura combinada

In [31]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate

# Cargar los datos
df = pd.read_csv("data/stickers/train_preprocessed.csv")
df = df.sort_values('date')

# Identificar combinaciones únicas de país, producto y tienda
combinaciones = df[['country', 'product', 'store']].drop_duplicates()

# Parámetros
window_size = 60  # Tamaño de la ventana de ventas anteriores

# Almacenar secuencias para todas las series temporales
X_sales, X_current, y = [], [], []

df.dropna(inplace=True) # de momento los tiramos, ya veremos si se peude imputar

In [32]:
# Función para crear secuencias de ventas anteriores
def create_sales_sequences(sales, window_size):
    X = []
    for i in range(len(sales) - window_size):
        X.append(sales[i:i + window_size])
    return np.array(X)

# Procesar cada combinación única (serie temporal individual)
for _, combo in combinaciones.iterrows():
    subset = df[(df['country'] == combo['country']) & 
                (df['product'] == combo['product']) & 
                (df['store'] == combo['store'])]

    # Ordenar por fecha
    subset = subset.sort_values('date')

    if len(subset) == 0:
        continue

    # One hot encoding de variables categóricas
    subset = pd.get_dummies(subset, columns=['country', 'product', 'store'])

    # Extraer ventas y características del día actual
    sales = subset['num_sold'].values
    current_features = subset.drop(columns=['date', 'num_sold']).values

    # Normalizar ventas y características
    scaler_sales = MinMaxScaler()
    scaler_features = MinMaxScaler()
    
    scaled_sales = scaler_sales.fit_transform(sales.reshape(-1, 1))
    scaled_features = scaler_features.fit_transform(current_features)

    # Crear secuencias de ventas anteriores
    X_seq = create_sales_sequences(scaled_sales, window_size)
    X_cur = scaled_features[window_size:]  # Características del día actual correspondientes
    y_seq = scaled_sales[window_size:]  # Valores de ventas esperados

    # print("X SEQ:",X_seq)
    # print("CURRENT:",X_cur)
    # print("Y SEQ:",y_seq)

    # Asegurar que las dimensiones coincidan
    if len(X_seq) == len(X_cur) == len(y_seq):
        X_sales.extend(X_seq)
        X_current.extend(X_cur)
        y.extend(y_seq)

# Convertir listas a arrays numpy
X_sales = np.array(X_sales)
X_current = np.array(X_current)
y = np.array(y)

# Dividir en conjuntos de entrenamiento, validación y prueba
train_size = int(len(X_sales) * 0.8)
X_sales_train, X_sales_test = X_sales[:train_size], X_sales[train_size:]
X_current_train, X_current_test = X_current[:train_size], X_current[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# VALIDACIÓN
# train_size = int(len(X_sales_train) * 0.8)
# X_sales_train, X_sales_val = X_sales[:train_size], X_sales_train[train_size:]
# X_current_train, X_current_val = X_current_train[:train_size], X_current_train[train_size:]
# y_train, y_val = y_train[:train_size], y_train[train_size:]


# Construir el submodelo LSTM para las ventas anteriores
input_sales = Input(shape=(window_size, 1))
lstm_out = LSTM(50, activation='relu')(input_sales)

# Construir el submodelo denso para las características actuales
input_current = Input(shape=(X_current_train.shape[1],))
dense_out = Dense(32, activation='relu')(input_current)

# Combinar las salidas de ambos submodelos
combined = Concatenate()([lstm_out, dense_out])
output = Dense(1)(combined)

# Definir el modelo completo
model = Model(inputs=[input_sales, input_current], outputs=output)
model.compile(optimizer='adam', loss='mse')

# Entrenar el modelo
model.fit([X_sales_train, X_current_train], y_train, epochs=5, batch_size=32, validation_split=0.2)

# Hacer predicciones en el conjunto de prueba
predictions = model.predict([X_sales_test, X_current_test])

# Invertir la normalización para obtener los valores originales
predictions = scaler_sales.inverse_transform(predictions)

Epoch 1/5
4320/4320 ━━━━━━━━━━━━━━━━━━━━ 94s 21ms/step - loss: 0.0103 - val_loss: 0.0040
Epoch 2/5
4320/4320 ━━━━━━━━━━━━━━━━━━━━ 95s 22ms/step - loss: 0.0034 - val_loss: 0.0035
Epoch 3/5
4320/4320 ━━━━━━━━━━━━━━━━━━━━ 79s 18ms/step - loss: 0.0033 - val_loss: 0.0036
Epoch 4/5
4320/4320 ━━━━━━━━━━━━━━━━━━━━ 58s 13ms/step - loss: 0.0033 - val_loss: 0.0033
Epoch 5/5
4320/4320 ━━━━━━━━━━━━━━━━━━━━ 58s 13ms/step - loss: 0.0032 - val_loss: 0.0033
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step


In [36]:
# evaluar predicciones
from sklearn.metrics import mean_squared_error, mean_absolute_error
mean_squared_error(predictions,y_test)

406029.30675133306